## **GUIA 2**



## **La ecuación de Burgers**





La ecuación de Burgers

\begin{equation}
    \frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} - \nu \frac{\partial^2 u}{\partial x^2} = 0.                          
\end{equation}

Forma parte de un grupo de ecuaciones no-lineales en derivadas parciales
"universales" que aparecen en muchos contextos.

En rigor, fue introducida en 1915 por Bateman [1], quien propuso una
solución de estado estacionario para el problema. Más
tarde, en 1948, el físico holandés J. M. Burgers, propuso
esta ecuación como modelo matemático de turbulencia [2,
3] y debido a su amplia contribución, la ecuación lleva su
nombre. A pesar de ser una versión simplificada de la ecuación de Navier Stokes,
se caracteriza por tener un término
dependiente del tiempo, un término de advección y un
término de difusión, con lo cual guarda la no-linealidad y las principales características de esta.
Se utiliza muchas veces como modelo de juguete para testear implementaciones numéricas de flujos turbulentos.

En su forma mas general (con el término de difusión) la ecuación es del tipo parabólica, mientras que en el caso ideal (invisido) es una ecuación hiperbólica. En ambos casos, resulta útil buscar leyes de conservación, de camino a las soluciones.

[1] Bateman, H., Some recent researches on the motion of
fluids, Monthy Wather Rev. 43, 163-170 (1915).

[2] Burgers, J. M., A mathematical model illustrating the
theory or turbulence, Advances in Applied Mechanics 1,
171-199 (1948).

[3] Burgers, J. M., Mathematical examples illustrating
relations occuring in the theory of turbulent fluid motion,
Trans. Roy. Neth. Acad. Sci. Amsterdam 1. 17, 1-53
(1939).

## **Ejercicio 1**


$\bf a)$ Mostrar que esta ecuación conserva el momento incluso para $\nu \neq 0$.

$\bf b)$ A partir de la ecuación de balance mostrá la conservación local de la energía. Bajo que condiciones se conserva la energía global?

$\bf c)$ Explica la aparición de "frentes" a partir de la ecuación para la evolución de los gradientes. (*Ayuda: esto puede hacerce derivando la ecuación de Burgers con respecto de x*)

$\bf d)$ Mediante la sustitución: $u(x,t) = -2\nu \partial_x\phi/\phi$ (transformación de Hopf) es posible hallar la solución exacta de (1). Escribí la ecuación. Que tipo de soluciones tiene? Describí la aparición de frentes en términos de estas soluciones.



---
**Resolución aqui**


---



### a)
Para observar la conservación del momento en la Ecuación de Burgers podemos reescribirla como:
$$\frac{\partial u}{\partial t} + \frac{\partial}{\partial x}\left(\frac{u^2}{2} - \nu \frac{\partial u}{\partial x}\right) = 0.$$
Luego, si integramos respecto de $x$ en el intervalo $[a,b]$ obtenemos:
$$\int\limits_a^b \frac{\partial u}{\partial t} \,\mathrm{d}x + \int\limits_a^b \frac{\partial}{\partial x}\left(\frac{u^2}{2} - \nu \frac{\partial u}{\partial x}\right) \,\mathrm{d}x = 0,$$
$$\Rightarrow \frac{\partial}{\partial t}\int\limits_a^b u \,\mathrm{d}x + \left[\frac{u^2}{2} - \nu \frac{\partial u}{\partial x}\right]_a^b = 0.$$
Tomando ahora que $u(a,t) = u(b,t)$ (condiciones de borde comptabiles con un flujo periódico o con extremos cerrados) se tiene que el segundo término se anula, obteniendo:
$$\frac{\partial}{\partial t}\int\limits_a^b u \,\mathrm{d}x = 0.$$
Luego, como $\int_a^b u \,\mathrm{d}x = 0$ puede sumarse el término $u \frac{\partial}{\partial x} \int_a^b u \, \mathrm{d}x$ a la ecuación sin alterarla, obteniendo la derivada total de la integral:
$$\frac{\mathrm{d}}{\mathrm{d}t}\int\limits_a^b u \,\mathrm{d}x = 0,$$
siendo la integral en el lado izquierdo la cantidad de momento en el intervalo $[a,b]$.

### b)
Para mostrar la conservación de la energía multiplico la ecuación de Burgers por $u$ y reescribo la ecuación como:
$$
\begin{align*}
 u\frac{\partial u}{\partial t} + u^2\frac{\partial}{\partial x}u & = & \nu u \frac{\partial^2 u}{\partial x^2} \\
 \iff \frac{\partial}{\partial t}\!\left(\frac{u^2}{2}\right) + u\frac{\partial}{\partial x}\!\left(\frac{u^2}{2}\right) & = & \nu u \frac{\partial^2 u}{\partial x^2} \\
 \iff \frac{\mathrm{d}}{\mathrm{d}t}\!\left(\frac{u^2}{2}\right) & = & \nu u \frac{\partial^2 u}{\partial x^2} \\
\end{align*}.$$
Observando que $\frac{u^2}{2}$ es la energía, se tiene que hay una perdida de energía local debido a la difusión, por lo que solo se conservará en el caso de que no haya difusión ($\nu = 0$). Ahora, para estudiar la conservación global de la energía, queremos ver si la integral en el recinto (entre $a$ y $b$) de $\frac{u^2}{2}$ varía en el tiempo. Para ello integramos la segunda ecuación de arriba en el intervalo $[a, b]$:
$$\int\limits_a^b \frac{\partial}{\partial t} \left(\frac{u^2}{2}\right) \!\mathrm{d}x + \int\limits_a^b u\frac{\partial}{\partial x} \left(\frac{u^2}{2}\right) \!\mathrm{d}x = \int\limits_a^b \nu u \frac{\partial^2 u}{\partial x^2} \!\mathrm{d}x,$$
$$\iff \frac{\partial}{\partial t} \int\limits_a^b \left(\frac{u^2}{2}\right) \!\mathrm{d}x + \int\limits_a^b u^2\frac{\partial u}{\partial x} \!\mathrm{d}x = \int\limits_a^b \nu u \frac{\partial^2 u}{\partial x^2} \!\mathrm{d}x,$$
$$\iff \frac{\partial}{\partial t} \int\limits_a^b \left(\frac{u^2}{2}\right) \!\mathrm{d}x + \int\limits_a^b \frac{\partial}{\partial x} \left(\frac{u^3}{3}\right) \!\mathrm{d}x = \int\limits_a^b \nu u \frac{\partial^2 u}{\partial x^2} \!\mathrm{d}x,$$
$$\iff \frac{\partial}{\partial t} \int\limits_a^b \left(\frac{u^2}{2}\right) \!\mathrm{d}x + \left[\frac{u^3}{3}\right]_a^b = \int\limits_a^b \nu u \frac{\partial^2 u}{\partial x^2} \!\mathrm{d}x,$$

Al mismo tiempo, el término de la derecha puede desarrollarse por partes según:
$$\int\limits_a^b \nu u \frac{\partial^2 u}{\partial x^2} \!\mathrm{d}x = \left[\nu u \frac{\partial u}{\partial x}\right]_a^b - \int\limits_a^b \nu \left(\frac{\partial u}{\partial x}\right)^2 \!\mathrm{d}x.$$


## **Ejercicio 2: Burgers con diferencias finitas**

$\bf a)$ Derivá la expresión para resolver numéricamente la ecuación de Burgers mediante un método de diferencias finitas con un esquema de Euler adelantado como integrador temporal.

$\bf b)$ Implementá numericamente esta expresión para resolver la ecuación en un dominio $[0,2\pi)$ considerando los siguientes parámetros: $\nu = 1\times 10^{-2}$ con $N_x = 512$ puntos de grilla y $\Delta t = 1\times10^{-3}$. Utilizá distintas condiciones iniciales del tipo:

$$ u_0 = sen(ax)$$
$$ u_0 = cos(x) + cos(bx) + sen(ax) $$

con a, b constantes arbitrarias.

$\bf c)$ De qué depende la aparición de los frentes? Siempre se ven? Graficá la solución obtenida ($u(x,t)$) para distintos tiempos sucesivos.

$\bf d)$ Qué sucede si comienzo con una velocidad inicial nula?




---

**Resolucion aqui**

---



**Pueden usar las siguientes líneas para implementar el método numérico**

In [ ]:
#Defino lo que voy a necesitar

nx = 512
nt = 1000
dx = 2 * np.pi / (nx - 1)
nu = .01
dt = 1e-3

x = np.linspace(0,2*np.pi,nx,endpoint=False) # Coordenada espacial en [0,2*pi)
t = np.arange(nt)*dt


In [ ]:
u = np.zeros([nx, nt])  # Array con la velocidad
u[0, :] = 0  # condición de contorno en x=0
u[nx-1, :] = 0  #  condición de contorno en  x=xmax


u[:, 0] = # alguna condicion inicial


for n in range(nt-1):
    u[1:nx-1, n+1] = (u[1:nx-1, n] +
                      dt * (nu * (u[2:nx, n] - 2 *...   #completar para integrar (fijate como es la expresión que te pedimos)

## **Ejercicio 3: Burgers con métodos pseudoespectrales**

Implementá un método de Fourier con condiciones de contorno periódicas en el dominio $x \in [0,2\pi)$ para resolver numericamente la ecuación de Burgers, considerando las mismas condiciones iniciales del ejercicio anterior y utilizando un método de Runge Kutta de orden dos como integrador temporal.

$\bf a)$ Integrá hasta $t_f = 4$ considerando los siguientes parámetros: $\nu = 0$, $N_x = 512$ puntos de grilla y $\Delta t = 1\times10^{-3}$, verificá cualitativamente la solución hallada y estudia la conservación de la energía. *Ayuda: podes calcular la energía (a menos de una constante) dado un arreglo* $u[t,x]$ como `E = np.mean(u**2, axis=1)`

$\bf b)$ Considerando los siguientes parámetros: $\nu = 1\times 10^{-2}$ con $N_x = 512$ puntos de grilla y $\Delta t = 1\times10^{-3}$, graficá la solución obtenida ($u(x,t)$) hasta que la amplitud inicial cae a la mitad. Describi
cualitativamente la dinámica.

$\bf c)$ Que sucede si se  aumenta la viscocidad a $\nu = 1\times 10^{-1}$? Y si se la disminuye a $\nu = 1\times 10^{-3}$?.



---

**Resolución aquí**


---






## **Bloques para la integración temporal**

Pueden completar el siguiente bloque para implementar la evolcuión temporal de la PDE con RK de orden dos y luego el método espectral

In [ ]:
#Defino mis puntos en el espacio real y los modos para el espacio de Fourier
x = np.linspace(0,2*np.pi,N,endpoint=False) # Coordenada espacial en [0,2*pi)
t = np.arange(step)*dt                      # Tiempo
k = np.arange(0,N/2+1)                      # Números de onda ordenados como en la FFT

In [ ]:
def evol(u,k,nu,N,dt):
    # Evoluciona en el tiempo la PDE usando el método pseudoespectral y Runge-Kutta de segundo orden
    f  = np.fft.rfft(u)     #vaya al espacio de Fourier
    ft = f
    for ord in [2,1]:       # Bucle para el Runge-Kutta
        D   = dt/ord
        fux = compute_advection(f,k,N)
        f   =                # escriba aquí su integrador temporal
        f[int(N/3):] =     # Dealiasing (eliminemos modos espúreos!)
    out = np.fft.irfft(f)  #Vuelva del espacio de Fourier
    return out

def compute_advection(f,k,N):
    # Calcula v*vx usando el método pseudoespectral y devuelve el resultado en el espacio Fourier
    fx  = 1j*k*f                      #derivamos
    v   =  np.fft.irfft(f)            # vuelva para poder multiplicar!
    vx  =                             # eso de arriba
    v   = v*vx
    out = np.fft.rfft(v)
    return out

## **Y ahora la integración numérica!**

In [ ]:
out = np.zeros((N,step))
out[:,0] = u
for i in np.arange(step-1): # Evolución temporal
    out[:,i+1] = evol()     #Completa para integrar (mira el bloque anterior)

## **Ejercicio 4: Burgers con PINNS**

Integre la ecuación de Burgers usando PINNs. En este caso, vamos a utilizar las PINNs como un integrador a partir de las condiciones iniciales.

Para ello, vamos a partir de una condición inicial $u_0(x,0) = \sin ( x)$ con $x \in [0,2 \pi]$. Además, vamos a poner condiciones de contorno tal que $u(0,t) = u(2 \pi,t) = 0$. Estos van a ser los datos que tenemos medidos del problema.


a) Usando la condición inicial descrita y con $\nu = 0$ use una PINN para integrar para $t \in [0,1]$. Grafique la solución de la red para distintos tiempos. Grafique $u(x,t)$ completo en todos los puntos donde evalua la física como una imagen. Interprete lo que ve. ¿Qué diferencias encuentra con lo visto en otros ejercicios? ¿Se cumple la conservación para la integral primera? [importante: Seleccione los parámetros por su cuenta]

b) Repita el punto $a$ pero con $\nu = 1 \times 10^{-2}$. Grafique para distintos tiempos y la matriz de  $u(x,t)$. Compare con la solución obtenida con métodos pseudoespectrales. Describa las diferencias que encuentra.

Importamos las librerías correspondientes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import scipy
from torch.autograd import Variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Definimos la clase MLP

In [ ]:
class MLP(torch.nn.Module):
    """
    Multilayer perceptron (MLP) // Perceptríon Multicapa .

    Esta clase define una red neuronal feedforward con múltiples capas ocultas
    lineales, funciones de activación tangente hiperbólica en  las capas ocultas
    y una salida lineal.

    Args:
        sizes (lista): Lista de enteros que especifica el número de neuronas en
        cada capa. El primer elemento debe coincidir con la dimensión de entrada
        y el último con la dimensión de salida.

    Atributos:
        capas (torch.nn.ModuleList): Lista que contiene las capas lineales del MLP.

    Métodos:
        forward(x): Realiza una pasada hacia adelante a través de la red MLP.

    Ejemplo:
        tamaños = [entrada_dim, oculta1_dim, oculta2_dim, salida_dim]
        mlp = MLP(tamaños)
        tensor_entrada = torch.tensor([...])
        salida = mlp(tensor_entrada)
    """
    def __init__(self,sizes):
        super().__init__()
        self.layers = torch.nn.ModuleList()
        for i in range(len(sizes)-1):
            self.layers.append(torch.nn.Linear(sizes[i],sizes[i+1]))
    def forward(self,x):
        h = x
        for hidden in self.layers[:-1]:
            h = torch.tanh(hidden(h))
        output = self.layers[-1]
        y = output(h)
        return y

Definimos las condiciones iniciales y condiciones de contorno

In [ ]:
#Condiciones en t = 0
t_data_init  =  #defina 100 puntos entre 0 y 1   No olvidar .view(-1,1)
x_data_init  =  #defina 100 puntos entre 0 y 2pi No olvidar .view(-1,1)
y_data_init  =  #defina la condicion inicial     No olvidar .view(-1,1)
input_init = torch.cat((x_data_init.flatten().view(-1,1), t_data_init.flatten().view(-1,1)), dim=1) #Junta las condiciones iniciales para que tengan la forma correcta

In [ ]:
#condiciones de contorno
t_data_cc    = torch.cat((torch.linspace(0,1,100), torch.linspace(0,1,100)), dim = 0).view(-1,1) #100 puntos temporales dos veces
x_data_cc    = 2*torch.pi* torch.cat((torch.zeros(100), torch.ones(100)), dim = 0).view(-1,1) #100 puntos en 0 y 100 en 1
y_data_cc    = torch.cat((torch.zeros(100), torch.zeros(100)), dim = 0 ).view(-1,1) #200 puntos que valen 0
input_cc = torch.cat((x_data_cc.flatten().view(-1,1), t_data_cc.flatten().view(-1,1)), dim=1) #Los juntamos para que pueda ser input la red

Definimos los puntos para evaluar la física. Estos deberán ser una grilla de 100x100 en el tiempo y el espacio.

In [ ]:
t_physics = #100 puntos de 0 a 1
x_physics = #100 puntos de 0 a 2 pi
x_grid, t_grid = torch.meshgrid(t_physics, x_physics, indexing='ij')
x_grid = x_grid[:,:,None].requires_grad_(True) #Agregamos una dimensión al final para que pueda ser input de la red
t_grid = t_grid[:,:,None].requires_grad_(True) #Agregamos una dimensión al final para que pueda ser input de la red
input_physics = torch.cat((x_grid, t_grid), dim=-1)

Defina la red y el optimizador. Se recomienda para empezar a explorar una red con $8$ capas ocultas con $20$ neuronas cada una. Use el peso relativo entre la física y lo demás como 1.

In [ ]:
#Definición de la red y optimizador

Complete con lo que sea necesario el loop de entrenamiento

In [ ]:
iterations =  #Numero de iteraciones
l =    #Peso relativo lambda

for epoch in range(iterations):
    optimizer.zero_grad()
    #Condiciones iniciales
    yh =      #Evalue las condiciones iniciales en la red
    loss1 =   #Defina este error como el MSE de la condición inicial
    #condiciones de contorno
    yh =       #Evalue las condiciones de contorno en la red
    loss2 =    #Defina este error como el MSE de las condiciones de contorno
    #Condiciones de la física
    yhp =      #Evalue las colocaciones de la física en la red
    dx  = torch.autograd.grad(yhp, x_grid, torch.ones_like(yhp), create_graph=True)[0]# computamos u_x
    dx2  = torch.autograd.grad(dx, x_grid, torch.ones_like(yhp), create_graph=True)[0]# computamos u_xx
    dt  = torch.autograd.grad(yhp, t_grid, torch.ones_like(yhp), create_graph=True)[0]# computamos u_t
    physics =  #Complete con la ecuación diferencial
    loss3 =    #Defina este error como el MSE de la ecuación diferencial en las colocaciones pesado por l
    loss = loss1 + loss2 + loss3  #Sumamos todos los errores
    loss.backward()
    optimizer.step()
    with torch.autograd.no_grad():
    	print(epoch,'CI',float(loss1), 'CC', float(loss2), 'Física:',float(loss3),"Traning Loss:",float(loss.data))

## **Ejercicio 5: Ecuación de Kardar-Parisi-Zhang** (Burgers potencial)

La ecuación de Burgers en forma potencial puede hallarse considerando $\overline{u} = \overline{\nabla}\phi$:

\begin{equation*}
    \frac{\partial \phi}{\partial t} + \frac{1}{2}(\overline{\nabla}\phi)^2 - \nu \nabla^2 \phi.
\end{equation*}

A partir de esto, el modelo de Kardar-Parisi-Zhang (KPZ) se obtiene mediante un cambio de variables simple para $\phi(x,t)$ y agregando difusión y ruido al azar:

\begin{equation*}
    \frac{\partial h}{\partial t} + \frac{\lambda^2}{2}(\overline{\nabla}h)^2 - \nu \nabla^2 h + \eta,
\end{equation*}

donde $\eta$ representa un ruido Gaussiano.

Dada la versión lineal de KPZ (Burgers potencial) con forzado aleatorio

\begin{equation*}
    \frac{\partial \phi}{\partial t} +  \frac{1}{2}(\frac{\partial \phi}{\partial x})^2-\nu \frac{\partial^2 \phi}{\partial x^2}+\eta
\end{equation*}

donde $\eta(x,t)$ es ruido aleatorio con valor medio nulo, se pide:

$\bf a).$ Integrá la ecuación con condiciones de contorno periódicas entre $x \in [0,2\pi)$, partiendo de una velocidad inicial nula y un ruido aleatorio con valor medio nulo. Como integrador temporal puede usarse Runge Kutta de orden 2.


$\bf b).$ Graficá la evolución en $(x,t)$, la evolcuión  para distintos t ($u(x,t)$ vs $t$) y zoom sucesivos para el ultimo tiempo. Describí cualitativamente la dinámica.

$\bf c)$ Cómo recupero Burgers? Graficá al menos una solución (alcanza con una condición inicial y una evolución de la misma para un tiempo) que sea como las que obtuviste en los ejercicios anteriores. Explicá (o escribí explicitamente en tu implementacion numérica) de que forma obtuviste esta solución.







## **Bloques para la integración temporal**

Pueden completar el siguiente bloque para la integración temporal.
Tengan en cuenta que pueden reciclar bastante del ejercicio anterior

In [ ]:
def evol(u,k,nu,N,dt):
    # Evoluciona en el tiempo la PDE usando el método pseudoespectral y Runge-Kutta de segundo orden
    f   = np.fft.rfft(u)
    ft  = f
    eta = compute_random(N) # Calculamos el ruido para cada iteración completa de Runge-Kutta
    for ord in [2,1]:       # Bucle para el Runge-Kutta
        D   = dt/ord
        fx  = 1j*k*f        # Derivada espacial de u en el espacio Fourier
        fnl = compute_vv(fx,N)
        f   =               #integrador temporal
        f[int(N/3):] = 0    # Dealiasing
    out = np.fft.irfft(f)
    return out

def compute_vv(f,N):
    # Calcula v^2 usando el método pseudoespectral y devuelve el resultado en el espacio Fourier

    return out

def compute_random(N):
    # Genera ruido Gaussiano con media nula y varianza unitaria en el espacio Fourier
    phase  = 2*np.pi*np.random.rand(int(N/2+1))
    ampl   = np.random.randn(int(N/2+1))
    out    =
    out[0] = 0.
    return out